In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [18]:
def Anfangsbedigungen(Nx,CFL,verbose=True):
    xmax = 10 
    xmin = 0
    dx = (xmax-xmin)/Nx 
    x = np.linspace(xmin, xmax, Nx) 
    dt = CFL*dx                     # Zeitschrittweite
    t_ende = 5                      # Endzeit
    Nt = int(t_ende/dt)             # Anzahl der Zeitschritte
    
    #Upwind-Verfahren
    c_positiv = 1*(dt/dx)
    c_negativ = 0
    
    #Lax-Friedrich / Lax-Wendroff-Verfahren
    c=dt/dx
    
    # Glatte Anfangsbedingung
    Uo_glatt = np.exp(-2.5*(x-2)**2)
    
    # Initialisierung der Zustandsmatrix
    U_glatt = np.zeros((Nt+1, Nx))
    U_glatt[0] = Uo_glatt
    
    # Unstetige Anfangsbedingung
    Uo_unstetig = np.where(np.logical_and(x>=1, x<=3), 1, 0)
    
    # Initialisierung der Zustandsmatrix
    U_unstetig = np.zeros((Nt+1, Nx))
    U_unstetig[0] = Uo_unstetig
    
    if verbose:
        return U_glatt, U_unstetig, c_positiv, c_negativ, Nt, Nx, x, dt #Upwind-Verfahren
    else:
        return U_glatt, U_unstetig, c, Nt, Nx,x,dt #Lax-Friedrich / Lax-Wendroff-Verfahren / Analytische Lösung (verbose=False)
    
def upwind_verfahren(Nx, CFL=1):    
    
    U_glatt, U_unstetig, c_positiv, c_negativ, Nt, Nx, x, dt = Anfangsbedigungen(Nx,CFL)
    
    #Lösung des beschriebenen Problems
    for n in range(Nt):
        for i in range(1, Nx-1):
            U_glatt[n+1, i] = U_glatt[n, i] - c_positiv*(U_glatt[n, i] - U_glatt[n, i-1]) + c_negativ*(U_glatt[n, i+1] - U_glatt[n, i])
            U_unstetig[n+1, i] = U_unstetig[n, i] - c_positiv*(U_unstetig[n, i] - U_unstetig[n, i-1]) + c_negativ*(U_unstetig[n, i+1] - U_unstetig[n, i])
        #Randbedingungen
        U_glatt[n+1, 0] = 0
        U_glatt[n+1, Nx-1] = 0
        U_unstetig[n+1, 0] = 0
        U_unstetig[n+1, Nx-1] = 0
            
    return U_glatt, U_unstetig, x, dt
# Frage: Es ist noch nicht ganz klar mit den Randbedingungen

In [20]:
N = 100 
CFL = 0.9

U_glatt, U_unstetig, x, dt = upwind_verfahren(N, CFL)
print(len(x))

100


In [28]:
#Numerische Flussfunktion - Bestimmen (nur das momentan)
def numFluss_LAX_FRIEDRICH(dx,dt,U):
    
    #rechtseitiger Fluss (F_j+1/2)
    F_j12_r = np.zeros((len(U),len(U[0])))
    #linker Fluss (F_j-1/2)
    F_j12_l = np.zeros((len(U),len(U[0])))
    
    #Funktion F definieren (Matrix mit h und u) (siehe Skript 3 Implementierung)
    #Matrix H bzw. H°, Matrix HU bzw. HU° definieren (siehe Skript 3 Implementierung)
    
    
    for i in range(1,len(U[0])-1):
        F_j12_r[:,i] = 0.5*(U[:,i] + U[:,i-1]) - 0.5*c*(U[:,i] - U[:,i-1])
        F_j12_l[:,i] = 0.5*(U[:,i] + U[:,i+1]) - 0.5*c*(U[:,i+1] - U[:,i])
    
    #Andere Verfahren auch probieren, alle steht im Skript 
        
    return F_j12_r, F_j12_l
    

F = numFluss_LAX_FRIEDRICH(x,dt,U_glatt) #dx muss noch außerhalb der Funktion Anfangsbedingungen bestimmt werden
print(F.shape)
print(max(x)/len(x)) #-----------> dx

#Zeitschrittweite durch CFL Bedingung für Systeme bestimmen (Lax-Friedrich)
## Jacobimatrix von F bestimmen 
## Eigenwerte der Jacobimatrix bestimmen
## dt = CFL*dx/max(Eigenwerte)

#Frage: kann man Dokumente in Jupyter Notebook verbinden?, damit der Code nicht so lang wird
#Muss man die komponenten in einzelnen Richtungen betrachten? ansonsten funktioniert nicht mit der Vektoriellen Form von h und hu. Skript Seite 34 Gleichung 3.2
#Wie beeinfleusst die Anfangsbedingung die Lösung? (siehe Skript Seite 34)



(56, 100)
0.1
